In [1]:
import math
import numpy as np
np.float = float
np.int   = int
body_imu = [1.72, -15.86, -0.08]
crotch_imu = [1.86, -14.88, 0.57]
# body_imu = [-1.10, -1.36, 0.90]
# crotch_imu = [-1.01, -0.62, -14.12]
body_imu_rad    = np.deg2rad(body_imu)
crotch_imu_rad  = np.deg2rad(crotch_imu)
# --- helper 함수들 ---
#회전 행렬 만들기
#x축으로 roll, y축으로 pitch, z축으로 yaw
def rpy_to_mat(roll, pitch, yaw): 
    
    Rx = np.array([[1, 0, 0],
                   [0, math.cos(roll), -math.sin(roll)],
                   [0, math.sin(roll),  math.cos(roll)]])
    Ry = np.array([[ math.cos(pitch), 0, math.sin(pitch)],
                   [0, 1, 0],
                   [-math.sin(pitch), 0, math.cos(pitch)]])
    Rz = np.array([[math.cos(yaw), -math.sin(yaw), 0],
                   [math.sin(yaw),  math.cos(yaw), 0],
                   [0, 0, 1]])
    return Rz @ Ry @ Rx


def axis_angle_to_mat(axis, angle):
    axis = np.array(axis) / np.linalg.norm(axis) #단위 벡터로 만들기
    x,y,z = axis #축을 x,y,z로 나누기
    c, s = math.cos(angle), math.sin(angle) #삼각함수 각도 구하기
    C = 1-c #1-cos 저장
    # Rodrigues 회전 공식 적용
    return np.array([
      [x*x*C + c,   x*y*C - z*s, x*z*C + y*s],
      [y*x*C + z*s, y*y*C + c,   y*z*C - x*s],
      [z*x*C - y*s, z*y*C + x*s, z*z*C + c  ]
    ])

In [2]:
import json
from urdfpy import URDF

ROOT_PATH = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001'
# 1. URDF와 data.json 파일 경로
URDF_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1_inspire_hand_description/g1_29dof_rev_1_0_with_inspire_hand_FTP.urdf'    # URDF가 담긴 .txt 파일
DATA_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'   # data.json 파일


finger_map = {'thumb':5, 'thumb2':4, 'index':3, 'middle':2, 'ring':1, 'pinky':0}

LEFTRIGHT = 'left'
WHICH_FINGER = 'index'
FRAME = 942

names = [LEFTRIGHT, 'shoulder', 'elbow', 'wrist', 'pitch', 'roll', 'yaw']   #얘는 list로 비교하면서 바꿨고 (아래 for i, joint in enumerate(joints) 부분에서)
handmap = {
                f'k{LEFTRIGHT.capitalize()}HandThumbRotation' : f'{LEFTRIGHT}_thumb_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandThumbBend' : f'{LEFTRIGHT}_thumb_3_joint',
                f'k{LEFTRIGHT.capitalize()}HandIndex' : f'{LEFTRIGHT}_index_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandMiddle' : f'{LEFTRIGHT}_middle_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandRing' : f'{LEFTRIGHT}_ring_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandPinky' : f'{LEFTRIGHT}_little_1_joint',
            }

# 2. URDF 로드
robot = URDF.load(URDF_PATH)

# 3. data.json 로드
with open(DATA_PATH, 'r') as f:
    data = json.load(f)


joints  = data['info']['joint_names'][f'{LEFTRIGHT}_arm']  # 7개 joint names list       여기부터

for i, joint in enumerate(joints):
    tmp_name = ""

    joint = joint[1:]
    for name in names:
        find_res = joint.lower().find(name)
        if find_res == -1:
            continue
        else:
            tmp_name = tmp_name + joint[:find_res+len(name)] + '_'
            joint = joint[find_res+len(name):]
    
    joints[i] = (tmp_name[:-1] + '_joint').lower()

# 4. idx=0의 left_arm 관절각(qpos)과 joint 이름 목록 가져오기
qpos    = data['data'][FRAME]['states'][f'{LEFTRIGHT}_arm']['qpos']                         #여기까지는 arm을 뽑는부분

hand_qpos  = data['data'][FRAME]['states'][f'{LEFTRIGHT}_hand']['qpos']                 #여기부터
hand_joints = data['info']['joint_names'][f'{LEFTRIGHT}_hand']
hand_joints = [handmap[x] for x in hand_joints]                                       #여기까지는 hand joints 정리해두는부분 이중에 한개만 뽑아서 넣을거임

# 5. joint name → angle 매핑
q_dict = {name: angle for name, angle in zip(joints, qpos)}

q_dict[hand_joints[finger_map[WHICH_FINGER]]] = hand_qpos[finger_map[WHICH_FINGER]]       #hand q_dict에 추가

T_world_base = np.eye(4)
T_world_base[:3,:3] = rpy_to_mat(*body_imu_rad)
T_world_base[:3,3] = [0,0,crotch_imu_rad[2]]
# 6. forward kinematics 계산  
# ee_link = f'{LEFTRIGHT}_wrist_yaw_link'     # wrist뽑을때 uncomment, hand 뽑을때 comment
ee_link = hand_joints[finger_map[WHICH_FINGER]][:-6]                    # hand 뽑을때 uncomment, wrist 뽑을때 comment
print(ee_link)
# link_fk()는 각 링크별로 월드 기준 4×4 변환행렬을 반환
fk = robot.link_fk(q_dict)
# for link_obj, T in fk.items():
#     print(link_obj.name)
# T_ee_world = fk[ee_link]   # (4,4) 행렬
ee_link_obj  = robot.link_map[ee_link]    # 이름→Link 객체 얻기
T_ee_world   = fk[ee_link_obj]                 # 올바르게 인덱싱!

# 7. 위치(translation) 추출
pos_ee_world = T_world_base@T_ee_world 
print('Left arm end-effector position in world frame:', pos_ee_world)


left_index_1
Left arm end-effector position in world frame: [[ 0.49976709  0.25524336  0.82769782  0.26168679]
 [ 0.55512993 -0.82791925 -0.07987792  0.18907581]
 [ 0.66487865  0.49940019 -0.55546002  0.30870893]
 [ 0.          0.          0.          1.        ]]


/home/scilab/anaconda3/envs/jy_tv/lib/python3.10/site-packages/urdfpy/urdf.py:2169: RuntimeWarning: invalid value encountered in divide
  value = value / np.linalg.norm(value)


In [3]:
# cam = robot['d435I_joint']
T_world_cam = np.eye(4)
T_world_cam[:3,:3] = rpy_to_mat(0, 0.8307767239493009, 0)
T_world_cam[:3, 3] = [0.0576235, 0.01753, 0.42987]   # :contentReference[oaicite:0]{index=0}


# 5) camera→left_wrist 변환
T_cam_ee = np.linalg.inv(T_world_cam) @ pos_ee_world

# 6) translation 성분이 바로 “카메라 기준” 손끝 좌표
pos_cam_ee = T_cam_ee[:3, 3]
print("Left wrist position in D435 camera frame:", pos_cam_ee)

Left wrist position in D435 camera frame: [0.2270724  0.17154581 0.06899243]


In [4]:
import cv2
import numpy as np

# 1) Intrinsics
width, height = 848, 480
fx = 426.7954
fy = 426.7954
# cx = width  / 2  # 424
# cy = height / 2  # 320
# fx=605.9278 
# fy=606.1917 
# cx = width  / 2  
# cy = height / 2  
cx = 414.0560
cy = 260.2649
# 2) pos_cam_ee: (X, Y, Z)  
#    X: 오른쪽(+), Y: 카메라 앞쪽(+), Z: 위쪽(+)
# print(pos_cam_ee)
# Xc =  pos_cam_ee[0]      
# Yc =  pos_cam_ee[1]      
# Zc =  pos_cam_ee[2]     
# Xc, Yc, Zc = T_cam_ee[:3,3]
X_urdf, Y_urdf, Z_urdf = pos_cam_ee

# 1) 축 재배열
Xc =  X_urdf
Yc = -Y_urdf
Zc = -Z_urdf
# 2) 투영
u = fx * (Yc / Xc) + cx
v = fy * (Zc / Xc) + cy

# 소수점→정수 픽셀 좌표
print(u,v)
pt = (int(round(u)), int(round(v)))
print(pt)
# 4) 이미지 읽고 표시  
img = cv2.imread("/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000942_color_0.jpg")
print(FRAME)
if img is None:
    raise RuntimeError("Unable to load image.")
cv2.circle(img, pt, 8, (255,0,0), thickness=-1)  # 빨간 점
cv2.putText(img, f"{pt}", (pt[0]+10, pt[1]),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

# 5) 결과 저장 (Jupyter에서 imshow 대신)
cv2.imwrite(f"projected_pointindex2{FRAME}.png", img)
print("Saved to projected_point.png")


91.62596439076509 130.58973241586406
(92, 131)
942
Saved to projected_point.png


In [5]:
import numpy as np
import json
import cv2
from pathlib import Path
from urdfpy import URDF
from scipy.optimize import least_squares

# =========================================================
# 경로 / 설정
# =========================================================
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  # 없으면 '' 또는 None
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'
OUT_DIR         = './joint_opt_vis'

# ======= 캘리브/시각화 프레임 =======
FRAMES_CALIB = [0,100,200,300,400,500,600,700,800,900,1000,1200,1300,1400,1500,1600,942]
FRAMES_VIS   = FRAMES_CALIB

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0,  0,  1]], dtype=float)
dist = np.zeros(5)

# ===== 초기 Extrinsics (camera ← world) : 이전 공동최적화 결과값을 초기치로 사용 =====
R_cw_init = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw_init = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

# 손목 오프셋 초기값 (손목 로컬 좌표계)
dL_init = np.array([-0.01958220,  0.03467778, -0.00843096], dtype=float)   # left
dR_init = np.array([ 0.00669406, -0.00187255,  0.03843392], dtype=float)   # right

# 로봇 바디 IMU (deg→rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])  # roll, pitch, yaw

# 가중치 (손목 2D > 손끝 2D)
W_WRIST = 1.0
W_TIP   = 0.7

# =========================================================
# 유틸
# =========================================================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    Kc = np.array([[0, -k[2], k[1]],
                   [k[2], 0, -k[0]],
                   [-k[1], k[0], 0]])
    return np.eye(3) + np.sin(theta)*Kc + (1-np.cos(theta))*(Kc@Kc)

def R_to_rodrigues(R):
    rvec, _ = cv2.Rodrigues(R.astype(np.float64))
    return rvec.flatten()

def to_h(T_or_R, t=None):
    if T_or_R.shape == (4,4):
        return T_or_R.copy()
    T = np.eye(4)
    T[:3,:3] = T_or_R
    if t is not None:
        T[:3,3] = t
    return T

# 클릭 도구 ------------------------------------------------
class ClickCollector:
    def __init__(self, scale=1.0):
        self.scale = scale
        self.pt = None

    def _cb(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.pt = (x/self.scale, y/self.scale)
            disp = param
            cv2.circle(disp, (x,y), 6, (0,0,255), -1)
            cv2.imshow(self.win, disp)

    def click_or_skip(self, img_path, label):
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(img_path)
        h, w = img.shape[:2]
        disp = cv2.resize(img, (int(w*self.scale), int(h*self.scale)))
        self.pt = None
        self.win = f'click_{label}'
        cv2.imshow(self.win, disp)
        cv2.setMouseCallback(self.win, self._cb, disp)
        print(f'[ {label} ]  {img_path}  — 클릭(좌클릭) 또는 s키로 건너뛰기')
        while True:
            k = cv2.waitKey(0) & 0xFF
            if k in (27, ord('s')):  # ESC or 's' -> skip
                cv2.destroyWindow(self.win)
                return None
            if self.pt is not None:
                cv2.destroyWindow(self.win)
                return np.array(self.pt, dtype=float)
            # otherwise keep waiting

# =========================================================
# 데이터/URDF 로드 및 맵
# =========================================================
print('[load] URDFs ...')
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print(f"[WARN] RIGHT_HAND_URDF 로드 실패: {e}. 오른손 손끝은 생략합니다.")
        right_hand = None

print('[load] data.json ...')
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
# 오른손 키 이름은 실제 data['info']['joint_names']['right_hand'] 확인 후 맞춰주세요.
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

def safe_qdict(names_list, qpos_list, name_to_joint):
    q = {}
    for n, v in zip(names_list, qpos_list):
        if n in name_to_joint:
            q[name_to_joint[n]] = v
        else:
            # 경고만 출력하고 무시
            pass
    return q

# =========================================================
# BODY URDF에서 wrist→hand 장착 변환 자동 추출
# =========================================================
def find_mount_transform(robot, wrist_link, prefer_side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            child_low = j.child.lower()
            if ('hand' in child_low) or ('finger' in child_low):
                cand.append(j)
    if not cand:
        return None, None
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            if prefer_side == 'left' and ('l_' in j.child.lower() or 'left' in j.child.lower()):
                picked = j; break
            if prefer_side == 'right' and ('r_' in j.child.lower() or 'right' in j.child.lower()):
                picked = j; break
    # urdfpy Joint.origin 은 4x4 ndarray
    return picked, picked.origin

jL, T_wrist_to_hand_L = find_mount_transform(robot, 'left_wrist_yaw_link', 'left')
jR, T_wrist_to_hand_R = find_mount_transform(robot, 'right_wrist_yaw_link', 'right')

if T_wrist_to_hand_L is None:
    print('[WARN] left wrist->hand mount not found. Use Identity.')
    T_wrist_to_hand_L = np.eye(4)
else:
    print('[MOUNT-LEFT] parent:', jL.parent, 'child:', jL.child)
    # print(T_wrist_to_hand_L)

if T_wrist_to_hand_R is None:
    print('[WARN] right wrist->hand mount not found. Use Identity.')
    T_wrist_to_hand_R = np.eye(4)
else:
    print('[MOUNT-RIGHT] parent:', jR.parent, 'child:', jR.child)
    # print(T_wrist_to_hand_R)

# =========================================================
# FK 모듈
# =========================================================
def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = { amap[n]: v for n,v in zip(names, qpos) if n in amap }
    fk = robot.link_fk(q)
    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    Rw = T_wrist[:3,:3]
    pw = T_wrist[:3,3]
    return Rw, pw

def left_index_tip_world(frame, d_off, dRvec, dT):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_qdict(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    T_hand_tip = fk_h[left_hand.link_map['L_index_tip']]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

def right_index_tip_world(frame, d_off, dRvec, dT):
    if right_hand is None:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_qdict(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)
    if 'R_index_tip' not in right_hand.link_map:
        return None
    T_hand_tip = fk_h[right_hand.link_map['R_index_tip']]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

# =========================================================
# 클릭 수집 (왼손목, 오른손목, 왼손끝, 오른손끝)
# =========================================================
collector = ClickCollector(scale=1.0)

obs = []  # 리스트 of dict per frame: {'frame':f,'LW':uv or None,'RW':..., 'LT':..., 'RT':...}
for fr in FRAMES_CALIB:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    print(f"\n[frame {fr}] 수집 시작")
    entry = {'frame': fr, 'LW': None, 'RW': None, 'LT': None, 'RT': None}
    try:
        uv = collector.click_or_skip(img_path, f'{fr}-LW(wrist-left)')
        entry['LW'] = uv
    except Exception as e:
        print('[skip LW]', e)
    try:
        uv = collector.click_or_skip(img_path, f'{fr}-RW(wrist-right)')
        entry['RW'] = uv
    except Exception as e:
        print('[skip RW]', e)
    try:
        uv = collector.click_or_skip(img_path, f'{fr}-LT(left index tip)')
        entry['LT'] = uv
    except Exception as e:
        print('[skip LT]', e)
    try:
        uv = collector.click_or_skip(img_path, f'{fr}-RT(right index tip)')
        entry['RT'] = uv
    except Exception as e:
        print('[skip RT]', e)
    obs.append(entry)

valid_cnt = sum( (e['LW'] is not None) or (e['RW'] is not None) or (e['LT'] is not None) or (e['RT'] is not None) for e in obs )
print(f"\n[collect] frames with any clicks: {valid_cnt}/{len(obs)}")
if valid_cnt < 4:
    raise RuntimeError('클릭 데이터가 너무 적습니다. 최소 4프레임 이상 권장')

# =========================================================
# 최적화 변수 구성
# x = [ rvec_cw(3), t_cw(3), dL(3), dR(3), rvec_L(3), dT_L(3), rvec_R(3), dT_R(3) ] => 24
# =========================================================
rvec_cw0 = R_to_rodrigues(R_cw_init)
x0 = np.concatenate([
    rvec_cw0, t_cw_init,
    dL_init, dR_init,
    np.zeros(3), np.zeros(3),  # rvec_L, dT_L
    np.zeros(3), np.zeros(3)   # rvec_R, dT_R
], axis=0)

# bounds (느슨하게)
# 각 회전은 약 ±30deg, 오프셋은 ±5cm, mount translation은 ±3cm
rot_bound = np.deg2rad(30.0)
trans_wrist_bound = 0.05
trans_mount_bound = 0.03
lb = np.array([
    -rot_bound, -rot_bound, -rot_bound,   # rvec_cw
    -1.0, -1.0, -1.0,                     # t_cw (m) — 넓게 허용
    -trans_wrist_bound, -trans_wrist_bound, -trans_wrist_bound,   # dL
    -trans_wrist_bound, -trans_wrist_bound, -trans_wrist_bound,   # dR
    -rot_bound, -rot_bound, -rot_bound,   # rvec_L
    -trans_mount_bound, -trans_mount_bound, -trans_mount_bound,   # dT_L
    -rot_bound, -rot_bound, -rot_bound,   # rvec_R
    -trans_mount_bound, -trans_mount_bound, -trans_mount_bound,   # dT_R
])
ub = -lb

# 잔차 함수 ------------------------------------------------
def residual(x):
    rvec_cw = x[0:3]
    t_cw    = x[3:6]
    dL      = x[6:9]
    dR      = x[9:12]
    rvec_L  = x[12:15]
    dT_L    = x[15:18]
    rvec_R  = x[18:21]
    dT_R    = x[21:24]

    R_cw = rodrigues_to_R(rvec_cw)
    R_cw_cv, _ = cv2.Rodrigues(R_cw.astype(np.float64))  # not used, keep compatibility

    res = []
    for e in obs:
        fr = e['frame']
        # 손목 좌표 (world)
        RwL, pwL = wrist_pose_world(fr, 'left')
        RwR, pwR = wrist_pose_world(fr, 'right')
        # wrist offset 적용 (로컬)
        pL = (RwL @ dL) + pwL
        pR = (RwR @ dR) + pwR

        # 투영 helper
        def proj(pw):
            pc = R_cw @ pw + t_cw
            X,Y,Z = pc
            if Z <= 0:
                return None
            u = fx*X/Z + cx
            v = fy*Y/Z + cy
            return np.array([u, v], dtype=float)

        # 손목 residual
        if e['LW'] is not None:
            uv = proj(pL)
            if uv is None:
                res += [1e3, 1e3]
            else:
                res += list(W_WRIST * (uv - e['LW']))
        if e['RW'] is not None:
            uv = proj(pR)
            if uv is None:
                res += [1e3, 1e3]
            else:
                res += list(W_WRIST * (uv - e['RW']))

        # 손끝 residual
        if e['LT'] is not None:
            p_tip_L = left_index_tip_world(fr, dL, rvec_L, dT_L)
            uv = proj(p_tip_L)
            if uv is None:
                res += [1e3, 1e3]
            else:
                res += list(W_TIP * (uv - e['LT']))
        if e['RT'] is not None:
            p_tip_R = right_index_tip_world(fr, dR, rvec_R, dT_R)
            if p_tip_R is not None:
                uv = proj(p_tip_R)
                if uv is None:
                    res += [1e3, 1e3]
                else:
                    res += list(W_TIP * (uv - e['RT']))
            # 오른손 URDF 미구현이면 잔차를 넣지 않음
    return np.array(res, dtype=float)

print('\n[opt] start least_squares ...')
res = least_squares(
    residual, x0, verbose=2,
    ftol=1e-9, xtol=1e-9, gtol=1e-9, max_nfev=400,
    loss='soft_l1', f_scale=3.0
)
print('\n===== Joint optimization result =====')
print('status:', res.status, res.message)
print('cost:', res.cost)

x = res.x
rvec_cw = x[0:3]; t_cw = x[3:6]
dL = x[6:9]; dR = x[9:12]
rvec_L = x[12:15]; dT_L = x[15:18]
rvec_R = x[18:21]; dT_R = x[21:24]
R_cw = rodrigues_to_R(rvec_cw)
print('R_cw:\n', R_cw)
print('t_cw:', t_cw)
print('dL:', dL)
print('dR:', dR)
print('dRvec_L:', rvec_L, '  dT_L:', dT_L)
print('dRvec_R:', rvec_R, '  dT_R:', dT_R)

# 최종 평균 reprojection error 보고
r = residual(x).reshape(-1,2)
err = np.linalg.norm(r, axis=1)
print('Reprojection error mean(px):', err.mean(), ' std(px):', err.std())

# =========================================================
# 시각화 저장
# =========================================================
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

def project_point(R_cw, t_cw, pw):
    pc = R_cw @ pw + t_cw
    X,Y,Z = pc
    if Z <= 0:
        return None, pc
    u = fx*X/Z + cx
    v = fy*Y/Z + cy
    return (int(round(u)), int(round(v))), pc

for fr in FRAMES_VIS:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    img = cv2.imread(img_path)
    if img is None:
        print('[WARN] image not found:', img_path)
        continue
    vis = img.copy()

    RwL, pwL = wrist_pose_world(fr, 'left')
    RwR, pwR = wrist_pose_world(fr, 'right')
    p_wL = (RwL @ dL) + pwL
    p_wR = (RwR @ dR) + pwR

    uv_wL, pc_wL = project_point(R_cw, t_cw, p_wL)
    uv_wR, pc_wR = project_point(R_cw, t_cw, p_wR)

    p_tL = left_index_tip_world(fr, dL, rvec_L, dT_L)
    uv_tL, pc_tL = project_point(R_cw, t_cw, p_tL) if p_tL is not None else (None, None)

    p_tR = right_index_tip_world(fr, dR, rvec_R, dT_R)
    uv_tR, pc_tR = project_point(R_cw, t_cw, p_tR) if p_tR is not None else (None, None)

    if uv_wL is not None:
        cv2.circle(vis, uv_wL, 8, (0,255,255), -1); cv2.putText(vis, 'LW', (uv_wL[0]+6, uv_wL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    if uv_wR is not None:
        cv2.circle(vis, uv_wR, 8, (255,255,0), -1); cv2.putText(vis, 'RW', (uv_wR[0]+6, uv_wR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
    if uv_tL is not None:
        cv2.circle(vis, uv_tL, 7, (0,0,255), -1); cv2.putText(vis, 'L_tip', (uv_tL[0]+6, uv_tL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    if uv_tR is not None:
        cv2.circle(vis, uv_tR, 7, (255,0,0), -1); cv2.putText(vis, 'R_tip', (uv_tR[0]+6, uv_tR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

    cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)
    out_path = str(Path(OUT_DIR) / f'vis_{fr}.png')
    cv2.imwrite(out_path, vis)
    print('Saved:', out_path)

print('\n완료!\n- 클릭 수가 많을수록(특히 손목+손끝 둘 다) 결과가 안정적입니다.'
      '\n- 오른손 손끝이 비어 있으면 RT 항은 자동으로 생략됩니다.'
      '\n- 결과가 과도하게 흔들리면 가중치(W_WRIST/W_TIP)나 bounds를 조절해 주세요.')

PARAM_PATH = './joint_opt_params.json'
params = {
    "R_cw": R_cw.tolist(),
    "t_cw": t_cw.tolist(),
    "dL":   dL.tolist(),
    "dR":   dR.tolist(),
    "rvec_L": rvec_L.tolist(),
    "dT_L":   dT_L.tolist(),
    "rvec_R": rvec_R.tolist(),
    "dT_R":   dT_R.tolist(),
}
with open(PARAM_PATH, 'w') as f:
    json.dump(params, f, indent=2)
print("Saved params to", PARAM_PATH)


[load] URDFs ...
[load] data.json ...
[MOUNT-LEFT] parent: left_wrist_yaw_link child: left_hand_palm_link
[MOUNT-RIGHT] parent: right_wrist_yaw_link child: right_hand_palm_link

[frame 0] 수집 시작
[ 0-LW(wrist-left) ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기


KeyboardInterrupt: 

In [ ]:
import numpy as np
import json, cv2
from pathlib import Path
from urdfpy import URDF

# ---------------- paths ----------------
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  # 없으면 '' 가능
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'

PARAM_PATH      = './joint_opt_params.json'   # 1단계에서 저장한 파일
OUT_DIR_TEST    = './test_vis'

# 테스트할 프레임들
FRAMES_TEST = [50, 75, 125, 225, 275, 325, 425, 525, 625, 725, 825, 925, 1025, 1125, 1225, 1325, 1425, 1525, 1625]

# camera intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649

# IMU
body_imu = np.deg2rad([1.72, -15.86, -0.08])

def rpy_to_mat(r,p,y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    K = np.array([[0,-k[2],k[1]],[k[2],0,-k[0]],[-k[1],k[0],0]])
    return np.eye(3) + np.sin(theta)*K + (1-np.cos(theta))*(K@K)

# ---- load params ----
with open(PARAM_PATH,'r') as f:
    P = json.load(f)
R_cw = np.array(P['R_cw'], dtype=float)
t_cw = np.array(P['t_cw'], dtype=float)
dL   = np.array(P['dL'],   dtype=float)
dR   = np.array(P['dR'],   dtype=float)
rvec_L = np.array(P['rvec_L'], dtype=float)
dT_L   = np.array(P['dT_L'],   dtype=float)
rvec_R = np.array(P['rvec_R'], dtype=float)
dT_R   = np.array(P['dT_R'],   dtype=float)

# ---- load URDFs ----
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print('[WARN] right hand urdf load fail:', e)

# joint maps
l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

with open(DATA_PATH,'r') as f:
    data = json.load(f)

def safe_qdict(names, qpos, mapping):
    out = {}
    for n,v in zip(names, qpos):
        if n in mapping:
            out[mapping[n]] = v
    return out

# mount transform from BODY urdf
def find_mount(robot, wrist_link, side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            c = j.child.lower()
            if 'hand' in c or 'finger' in c:
                cand.append(j)
    if not cand:
        return np.eye(4)
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            c = j.child.lower()
            if side=='left' and ('l_' in c or 'left' in c): picked = j; break
            if side=='right' and ('r_' in c or 'right' in c): picked = j; break
    return picked.origin

T_wrist_to_hand_L = find_mount(robot, 'left_wrist_yaw_link', 'left')
T_wrist_to_hand_R = find_mount(robot, 'right_wrist_yaw_link', 'right')

def wrist_pose_world(frame, side='left'):
    if side=='left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap = l_arm_map; wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap = r_arm_map; wrist_link = 'right_wrist_yaw_link'
    q = safe_qdict(names, qpos, amap)
    fk = robot.link_fk(q)
    T_world_base = np.eye(4); T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    return T_wrist[:3,:3], T_wrist[:3,3]

def left_tip_world(frame):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = dL
    R_delta = rodrigues_to_R(rvec_L)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT_L

    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_qdict(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    T_tip = fk_h[left_hand.link_map['L_index_tip']]

    T_w = np.eye(4); T_w[:3,:3]=Rw; T_w[:3,3]=pw
    Tw_tip = T_w @ T_off @ T_mount @ T_tip
    return Tw_tip[:3,3]

def right_tip_world(frame):
    if right_hand is None or 'R_index_tip' not in right_hand.link_map:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = dR
    R_delta = rodrigues_to_R(rvec_R)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT_R

    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_qdict(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)
    T_tip = fk_h[right_hand.link_map['R_index_tip']]

    T_w = np.eye(4); T_w[:3,:3]=Rw; T_w[:3,3]=pw
    Tw_tip = T_w @ T_off @ T_mount @ T_tip
    return Tw_tip[:3,3]

def project(R_cw, t_cw, pw):
    pc = R_cw @ pw + t_cw
    X,Y,Z = pc
    if Z <= 0: return None
    u = fx*X/Z + cx; v = fy*Y/Z + cy
    return int(round(u)), int(round(v))

Path(OUT_DIR_TEST).mkdir(parents=True, exist_ok=True)

for fr in FRAMES_TEST:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    img = cv2.imread(img_path)
    if img is None:
        print('[skip] image not found:', img_path); continue
    vis = img.copy()

    RwL, pwL = wrist_pose_world(fr, 'left')
    RwR, pwR = wrist_pose_world(fr, 'right')

    p_wL = pwL + RwL @ dL
    p_wR = pwR + RwR @ dR

    uv_wL = project(R_cw, t_cw, p_wL)
    uv_wR = project(R_cw, t_cw, p_wR)

    p_tL = left_tip_world(fr)
    uv_tL = project(R_cw, t_cw, p_tL) if p_tL is not None else None

    p_tR = right_tip_world(fr)
    uv_tR = project(R_cw, t_cw, p_tR) if p_tR is not None else None

    if uv_wL: cv2.circle(vis, uv_wL, 8, (0,255,255), -1); cv2.putText(vis,'LW',(uv_wL[0]+6,uv_wL[1]-6),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)
    if uv_wR: cv2.circle(vis, uv_wR, 8, (255,255,0), -1); cv2.putText(vis,'RW',(uv_wR[0]+6,uv_wR[1]-6),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,0),2)
    if uv_tL: cv2.circle(vis, uv_tL, 7, (0,0,255), -1); cv2.putText(vis,'L_tip',(uv_tL[0]+6,uv_tL[1]-6),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
    if uv_tR: cv2.circle(vis, uv_tR, 7, (255,0,0), -1); cv2.putText(vis,'R_tip',(uv_tR[0]+6,uv_tR[1]-6),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,0,0),2)

    cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)
    out_path = str(Path(OUT_DIR_TEST)/f'test_{fr}.png')
    cv2.imwrite(out_path, vis)
    print('Saved:', out_path)

print('Done.')


In [8]:
import numpy as np
import json, cv2
from pathlib import Path
from urdfpy import URDF
from scipy.optimize import least_squares

# ===========================
# 경로 / 설정
# ===========================
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  # 없으면 '' 또는 None
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'

OUT_DIR         = './ft_only_vis'
PARAM_SAVE_PATH = './ft_only_params.json'

# 보정에 사용할 프레임들 (원하는 만큼)
FRAMES_CALIB = [0,100,200,300,400,500,600,800,900,1000,1200,1300,1400,1500,1600,942]

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649
dist = np.zeros(5)

# 로봇 바디 IMU (deg→rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])  # roll, pitch, yaw

# 손목 오프셋을 변수로 쓸지 여부 (완전히 빼고 싶으면 False)
USE_WRIST_OFFSET = True

# 가중치 (양손 손끝 동일 가중)
W_TIP = 1.0

# 초기 외부파라미터 (대략값; 이전 결과 있으면 넣어두면 수렴이 쉽습니다)
R_cw_init = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw_init = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

# 손목 오프셋 초기값 (손목 로컬좌표) — 사용하지 않으면 0으로 둠
dL_init = np.array([-0.01958220,  0.03467778, -0.00843096], dtype=float) if USE_WRIST_OFFSET else np.zeros(3)
dR_init = np.array([ 0.00669406, -0.00187255,  0.03843392], dtype=float) if USE_WRIST_OFFSET else np.zeros(3)

# ===========================
# 유틸
# ===========================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    K = np.array([[0, -k[2], k[1]],
                  [k[2], 0, -k[0]],
                  [-k[1], k[0], 0]])
    return np.eye(3) + np.sin(theta)*K + (1-np.cos(theta))*(K@K)

def R_to_rodrigues(R):
    rvec, _ = cv2.Rodrigues(R.astype(np.float64))
    return rvec.flatten()

# ===========================
# 로드
# ===========================
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print(f"[WARN] RIGHT_HAND_URDF 로드 실패: {e}")

with open(DATA_PATH, 'r') as f:
    data = json.load(f)

# 조인트 맵
l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
    'kLeftHandThumbBend'    :'L_thumb_proximal_pitch_joint',
    'kLeftHandIndex'        :'L_index_proximal_joint',
    'kLeftHandMiddle'       :'L_middle_proximal_joint',
    'kLeftHandRing'         :'L_ring_proximal_joint',
    'kLeftHandPinky'        :'L_pinky_proximal_joint',
}
# 오른손 키 이름은 실제 data를 보고 맞춰주세요.
r_hand_map = {
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
    'kRightHandThumbBend'    :'R_thumb_proximal_pitch_joint',
    'kRightHandIndex'        :'R_index_proximal_joint',
    'kRightHandMiddle'       :'R_middle_proximal_joint',
    'kRightHandRing'         :'R_ring_proximal_joint',
    'kRightHandPinky'        :'R_pinky_proximal_joint',
}

# 손끝 링크명 매핑
l_tip_links = {
    'LT':'L_thumb_tip',
    'LI':'L_index_tip',
    'LM':'L_middle_tip',
    'LR':'L_ring_tip',
    'LP':'L_pinky_tip',
}
# 오른손은 URDF에 맞게 수정 필요. 없으면 자동 추정 시도.
r_tip_links = {
    'RT':'R_thumb_tip',
    'RI':'R_index_tip',
    'RM':'R_middle_tip',
    'RR':'R_ring_tip',
    'RP':'R_pinky_tip',
}

def ensure_tip_link_names(hand, mapping, prefix_hint='R_'):
    """주어진 링크명이 없으면, hand.link_map의 키를 훑어 가장 비슷한 걸 찾아 채워줌."""
    keys = list(hand.link_map.keys()) if hand is not None else []
    for k,v in list(mapping.items()):
        if hand is None or v in keys:
            continue
        # fallback: contains
        cand = [name for name in keys if 'tip' in name.lower()]
        # finger keyword 우선
        kw = k[1:].lower()  # 'T','I','M','R','P'
        finger_map = {'t':'thumb','i':'index','m':'middle','r':'ring','p':'pinky'}
        if kw in finger_map:
            word = finger_map[kw]
            sub = [n for n in cand if word in n.lower()]
            if sub:
                mapping[k] = sub[0]
                continue
        # prefix_hint
        sub = [n for n in cand if prefix_hint.lower() in n.lower()]
        if sub:
            mapping[k] = sub[0]
    return mapping

# 오른손 링크명 보정
r_tip_links = ensure_tip_link_names(right_hand, r_tip_links, 'R_')

def safe_qdict(names, qpos, mapping):
    out = {}
    for n, v in zip(names, qpos):
        if n in mapping:
            out[mapping[n]] = v
    return out

# BODY URDF에서 wrist→hand 장착 변환
def find_mount(robot, wrist_link, side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            c = j.child.lower()
            if ('hand' in c) or ('finger' in c):
                cand.append(j)
    if not cand:
        return np.eye(4)
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            c = j.child.lower()
            if side=='left'  and ('l_' in c or 'left'  in c): picked = j; break
            if side=='right' and ('r_' in c or 'right' in c): picked = j; break
    return picked.origin

T_wrist_to_hand_L = find_mount(robot, 'left_wrist_yaw_link', 'left')
T_wrist_to_hand_R = find_mount(robot, 'right_wrist_yaw_link', 'right')

def wrist_pose_world(frame, side='left'):
    if side=='left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map; wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map; wrist_link = 'right_wrist_yaw_link'
    q = safe_qdict(names, qpos, amap)
    fk = robot.link_fk(q)
    T_world_base = np.eye(4); T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_w = T_world_base @ fk[ robot.link_map[wrist_link] ]
    return T_w[:3,:3], T_w[:3,3]

def left_tip_world(frame, label, d_off, rvec_mount, dT_mount):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(rvec_mount)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT_mount

    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_qdict(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)

    link_name = l_tip_links[label]
    if link_name not in left_hand.link_map:
        return None
    T_tip = fk_h[left_hand.link_map[link_name]]

    T_w = np.eye(4); T_w[:3,:3]=Rw; T_w[:3,3]=pw
    Tw_tip = T_w @ T_off @ T_mount @ T_tip
    return Tw_tip[:3,3]

def right_tip_world(frame, label, d_off, rvec_mount, dT_mount):
    if right_hand is None:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(rvec_mount)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT_mount

    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_qdict(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)

    link_name = r_tip_links.get(label, None)
    if (link_name is None) or (link_name not in right_hand.link_map):
        return None
    T_tip = fk_h[right_hand.link_map[link_name]]

    T_w = np.eye(4); T_w[:3,:3]=Rw; T_w[:3,3]=pw
    Tw_tip = T_w @ T_off @ T_mount @ T_tip
    return Tw_tip[:3,3]

def project_point(R_cw, t_cw, pw):
    pc = R_cw @ pw + t_cw
    X,Y,Z = pc
    if Z <= 0: 
        return None
    u = fx*X/Z + cx
    v = fy*Y/Z + cy
    return np.array([u,v], dtype=float)

# ===========================
# 클릭 수집 (손끝 10점만)
# ===========================
class ClickCollector:
    def __init__(self, scale=1.0):
        self.scale = scale
        self.pt = None
        self.win = None
    def _cb(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.pt = (x/self.scale, y/self.scale)
            disp = param
            cv2.circle(disp, (x,y), 6, (0,0,255), -1)
            cv2.imshow(self.win, disp)
    def click_or_skip(self, img_path, label):
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(img_path)
        h, w = img.shape[:2]
        disp = cv2.resize(img, (int(w*self.scale), int(h*self.scale)))
        self.pt = None
        self.win = f'click_{label}'
        cv2.imshow(self.win, disp)
        cv2.setMouseCallback(self.win, self._cb, disp)
        print(f'[ {label} ]  {img_path}  — 클릭(좌클릭) 또는 s키로 건너뛰기')
        while True:
            k = cv2.waitKey(0) & 0xFF
            if k in (27, ord('s')): # ESC or s -> skip
                cv2.destroyWindow(self.win)
                return None
            if self.pt is not None:
                cv2.destroyWindow(self.win)
                return np.array(self.pt, dtype=float)

labels_left  = ['LT','LI','LM','LR','LP']
labels_right = ['RT','RI','RM','RR','RP']

collector = ClickCollector(scale=1.0)
obs = []  # per frame dict: {'frame':f, 'LT':uv or None, ...}
for fr in FRAMES_CALIB:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    print(f'\n[frame {fr}] 클릭 수집 시작')
    entry = {'frame': fr}
    for lb in labels_left + labels_right:
        try:
            uv = collector.click_or_skip(img_path, f'{fr}-{lb}')
            entry[lb] = uv
        except Exception as e:
            print(f'[skip {lb}]', e)
            entry[lb] = None
    obs.append(entry)

valid_cnt = sum(any(entry[lb] is not None for lb in labels_left+labels_right) for entry in obs)
print(f'\n[collect] clicks in frames: {valid_cnt}/{len(obs)}')
if valid_cnt < 4:
    raise RuntimeError('클릭 데이터가 너무 적습니다. 최소 4프레임 이상 권장')

# ===========================
# 최적화 변수
# x = [ rvec_cw(3), t_cw(3), dL(3), dR(3), rvec_L(3), dT_L(3), rvec_R(3), dT_R(3) ]
# 손목 오프셋을 비활성화하려면 dL,dR를 0으로 고정하고 잔차에서 사용만 하도록 구현
# ===========================
rvec_cw0 = R_to_rodrigues(R_cw_init)
if USE_WRIST_OFFSET:
    x0 = np.concatenate([rvec_cw0, t_cw_init, dL_init, dR_init,
                         np.zeros(3), np.zeros(3),  # rvec_L, dT_L
                         np.zeros(3), np.zeros(3)], axis=0)
else:
    x0 = np.concatenate([rvec_cw0, t_cw_init,
                         np.zeros(3), np.zeros(3),  # dL,dR (고정용 자리)
                         np.zeros(3), np.zeros(3),
                         np.zeros(3), np.zeros(3)], axis=0)

# 느슨한 bounds
rot_bound = np.deg2rad(30.0)
trans_bound = 0.10   # t_cw
off_bound   = 0.02   # dL,dR ±2cm
mount_t_bound = 0.02

lb = np.array([
    -rot_bound, -rot_bound, -rot_bound,  # rvec_cw
    -trans_bound, -trans_bound, -trans_bound,  # t_cw
    -off_bound, -off_bound, -off_bound,   # dL
    -off_bound, -off_bound, -off_bound,   # dR
    -rot_bound, -rot_bound, -rot_bound,   # rvec_L
    -mount_t_bound, -mount_t_bound, -mount_t_bound,  # dT_L
    -rot_bound, -rot_bound, -rot_bound,   # rvec_R
    -mount_t_bound, -mount_t_bound, -mount_t_bound,  # dT_R
])
ub = -lb

if not USE_WRIST_OFFSET:
    # dL,dR를 0으로 고정
    lb[6:12] = 0.0
    ub[6:12] = 0.0
    x0[6:12] = 0.0

def residual(x):
    rvec_cw = x[0:3]
    t_cw    = x[3:6]
    dL      = x[6:9]
    dR      = x[9:12]
    rvec_L  = x[12:15]
    dT_L    = x[15:18]
    rvec_R  = x[18:21]
    dT_R    = x[21:24]
    R_cw = rodrigues_to_R(rvec_cw)

    res = []
    for e in obs:
        fr = e['frame']
        # 왼손 5개
        for lb in labels_left:
            uv_gt = e[lb]
            if uv_gt is None: 
                continue
            p_w = left_tip_world(fr, lb, dL, rvec_L, dT_L)
            if p_w is None:
                continue
            uv = project_point(R_cw, t_cw, p_w)
            if uv is None:
                res += [1000.0, 1000.0]
            else:
                res += list(W_TIP * (uv - uv_gt))
        # 오른손 5개
        for lb in labels_right:
            uv_gt = e[lb]
            if uv_gt is None:
                continue
            p_w = right_tip_world(fr, lb, dR, rvec_R, dT_R)
            if p_w is None:
                continue
            uv = project_point(R_cw, t_cw, p_w)
            if uv is None:
                res += [1000.0, 1000.0]
            else:
                res += list(W_TIP * (uv - uv_gt))
    return np.array(res, dtype=float)

print('\n[opt] start least_squares (tips only) ...')
res = least_squares(
    residual, x0, verbose=2,
    ftol=1e-9, xtol=1e-9, gtol=1e-9, max_nfev=600,
    loss='soft_l1', f_scale=3.0
)
print('\n===== Tips-only optimization result =====')
print('status:', res.status, res.message)
print('cost:', res.cost)

x = res.x
rvec_cw = x[0:3]; t_cw = x[3:6]
dL = x[6:9]; dR = x[9:12]
rvec_L = x[12:15]; dT_L = x[15:18]
rvec_R = x[18:21]; dT_R = x[21:24]
R_cw = rodrigues_to_R(rvec_cw)

print('R_cw:\n', R_cw)
print('t_cw:', t_cw)
print('dL:', dL, '  dR:', dR)
print('rvec_L:', rvec_L, ' dT_L:', dT_L)
print('rvec_R:', rvec_R, ' dT_R:', dT_R)

# 평균 reprojection error
r = residual(x).reshape(-1,2)
err = np.linalg.norm(r, axis=1)
print('Reprojection error mean(px):', err.mean(), ' std(px):', err.std())

# 파라미터 저장
params = dict(
    R_cw=R_cw.tolist(), t_cw=t_cw.tolist(),
    dL=dL.tolist(), dR=dR.tolist(),
    rvec_L=rvec_L.tolist(), dT_L=dT_L.tolist(),
    rvec_R=rvec_R.tolist(), dT_R=dT_R.tolist(),
    USE_WRIST_OFFSET=USE_WRIST_OFFSET
)
with open(PARAM_SAVE_PATH, 'w') as f:
    json.dump(params, f, indent=2)
print('Saved params to', PARAM_SAVE_PATH)

# ===========================
# 시각화 + Palm 계산
# palm = 5개 손끝 월드 좌표의 centroid
# ===========================
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

def tips_world_all(frame):
    ptsL = []
    for lb in labels_left:
        p = left_tip_world(frame, lb, dL, rvec_L, dT_L)
        if p is not None: ptsL.append(p)
    ptsR = []
    for lb in labels_right:
        p = right_tip_world(frame, lb, dR, rvec_R, dT_R)
        if p is not None: ptsR.append(p)
    return np.array(ptsL) if ptsL else None, np.array(ptsR) if ptsR else None

def project_uv(R_cw, t_cw, p):
    pc = R_cw @ p + t_cw
    X,Y,Z = pc
    if Z <= 0: return None
    return int(round(fx*X/Z + cx)), int(round(fy*Y/Z + cy))

for e in obs:
    fr = e['frame']
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    img = cv2.imread(img_path)
    if img is None:
        print('[WARN] image not found:', img_path); continue
    vis = img.copy()

    # tips draw
    for lb in labels_left:
        p = left_tip_world(fr, lb, dL, rvec_L, dT_L)
        if p is None: continue
        uv = project_uv(R_cw, t_cw, p)
        if uv is None: continue
        cv2.circle(vis, uv, 6, (0,0,255), -1)
        cv2.putText(vis, lb, (uv[0]+4, uv[1]-4), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

    for lb in labels_right:
        p = right_tip_world(fr, lb, dR, rvec_R, dT_R)
        if p is None: continue
        uv = project_uv(R_cw, t_cw, p)
        if uv is None: continue
        cv2.circle(vis, uv, 6, (255,0,0), -1)
        cv2.putText(vis, lb, (uv[0]+4, uv[1]-4), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

    # palm = centroid of 5 tips
    ptsL, ptsR = tips_world_all(fr)
    if ptsL is not None and len(ptsL)>=3:
        palmL = ptsL.mean(axis=0)
        uv = project_uv(R_cw, t_cw, palmL)
        if uv: 
            cv2.circle(vis, uv, 9, (0,255,255), 2)
            cv2.putText(vis, 'L_palm', (uv[0]+6, uv[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)

    if ptsR is not None and len(ptsR)>=3:
        palmR = ptsR.mean(axis=0)
        uv = project_uv(R_cw, t_cw, palmR)
        if uv:
            cv2.circle(vis, uv, 9, (0,255,255), 2)
            cv2.putText(vis, 'R_palm', (uv[0]+6, uv[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)

    cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)
    out_path = str(Path(OUT_DIR) / f'ft_only_{fr}.png')
    cv2.imwrite(out_path, vis)
    print('Saved:', out_path)

print('\n완료!')
print('- 손바닥 좌표는 5개 손끝의 중심을 사용했습니다.')
print('- URDF의 palm/base 링크로 정의하고 싶으면, 해당 링크의 변환을 wrist 기준으로 불러와서 동일하게 합성해주면 됩니다.')



[frame 0] 클릭 수집 시작
[ 0-LT ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-LI ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-LM ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-LR ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-LP ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-RT ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-RI ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/data

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import json
import cv2
from pathlib import Path
from urdfpy import URDF
from scipy.optimize import least_squares

# =========================================================
# 경로 / 설정
# =========================================================
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  # 없으면 '' 또는 None
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'

OUT_DIR         = './palm_tip_vis'
CLICKS_JSON     = './clicks_palm_tip.json'   # 클릭 저장/불러올 파일

# 사용할 프레임들
FRAMES_VIS   = [0,100,200,300,400,500,600,900,1300,1500,1600,942]

# ---- 클릭 수집 방식 ----
USE_GUI      = True    # GUI가 안되면 False
SAVE_CLICKS  = True    # 수집한 클릭을 JSON으로 저장
LOAD_CLICKS  = False   # 이전에 저장한 클릭을 불러와 재사용

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0,  0,  1]], dtype=float)
dist = np.zeros(5)

# ===== 카메라 Extrinsics (camera ← world) : 이전 공동최적화 결과 =====
R_cw = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

# 손목 오프셋 (손목 로컬 좌표계) — 공동최적화 결과
dL = np.array([-0.01958220,  0.03467778, -0.00843096], dtype=float)   # left
dR = np.array([ 0.00669406, -0.00187255,  0.03843392], dtype=float)   # right

# 손끝 장착 미세보정 (회전/이동) — 필요 없으면 0
dRvec_L = np.zeros(3)
dT_L    = np.zeros(3)
dRvec_R = np.zeros(3)
dT_R    = np.zeros(3)

# 손바닥 중심 오프셋(손 베이스 프레임에서) — 기본 0
pPalm_L = np.array([0.0, 0.0, 0.0], dtype=float)
pPalm_R = np.array([0.0, 0.0, 0.0], dtype=float)

# 로봇 바디 IMU (deg→rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])  # roll, pitch, yaw

# 색상
COL_LW = (0,255,255)   # 왼손목
COL_RW = (255,255,0)   # 오른손목
COL_LT = (0,0,255)     # 왼손끝
COL_RT = (255,0,0)     # 오른손끝
COL_LP = (0,255,0)     # 왼손바닥
COL_RP = (255,0,255)   # 오른손바닥

# =========================================================
# 유틸
# =========================================================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    Kc = np.array([[0, -k[2], k[1]],
                   [k[2], 0, -k[0]],
                   [-k[1], k[0], 0]])
    return np.eye(3) + np.sin(theta)*Kc + (1-np.cos(theta))*(Kc@Kc)

def project_point_world(p_world):
    """world 3D → pixel(+카메라 좌표)"""
    p_cam = R_cw @ p_world + t_cw
    X, Y, Z = p_cam
    if Z <= 0:
        return None, p_cam
    u = fx * (X / Z) + cx
    v = fy * (Y / Z) + cy
    return (int(round(u)), int(round(v))), p_cam

# ---------- JSON 직렬화 유틸 ----------
def clicks_to_jsonable(clicks: dict) -> dict:
    """numpy를 모두 list/float/None으로 변환"""
    out = {}
    for fr, entry in clicks.items():
        e2 = {}
        for k, v in entry.items():   # LW/RW/LT/RT/LP/RP
            if v is None:
                e2[k] = None
            else:
                e2[k] = np.asarray(v, dtype=float).tolist()
        out[str(fr)] = e2
    return out

def clicks_from_json(raw: dict) -> dict:
    """JSON(dict of lists/None)을 numpy로 변환"""
    out = {}
    for fr, entry in raw.items():
        e2 = {}
        for k, v in entry.items():
            e2[k] = (np.array(v, dtype=float) if v is not None else None)
        out[str(fr)] = e2
    return out

# ---------- 클릭 도구 ----------
class Clicker:
    def __init__(self, scale=1.0):
        self.scale = scale
        self.pt = None
        self.win = None

    def _cb(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.pt = (x/self.scale, y/self.scale)
            disp = param
            cv2.circle(disp, (x,y), 6, (0,0,255), -1)
            cv2.imshow(self.win, disp)

    def click_or_none(self, img_path, label):
        if not USE_GUI:
            print(f'[skip {label}] GUI 비활성화')
            return None
        img = cv2.imread(img_path)
        if img is None:
            print(f'[skip {label}] image not found: {img_path}')
            return None
        h, w = img.shape[:2]
        disp = cv2.resize(img, (int(w*self.scale), int(h*self.scale)))
        self.pt = None
        self.win = f'click_{label}'
        try:
            cv2.imshow(self.win, disp)
            cv2.setMouseCallback(self.win, self._cb, disp)
            print(f'[ {label} ] {img_path} — 좌클릭(선택) / s 또는 ESC 건너뛰기')
            while True:
                k = cv2.waitKey(0) & 0xFF
                if k in (27, ord('s')):  # ESC or 's'
                    cv2.destroyWindow(self.win)
                    return None
                if self.pt is not None:
                    cv2.destroyWindow(self.win)
                    return np.array(self.pt, dtype=float)
        except cv2.error as e:
            print(f'[skip {label}] OpenCV GUI error: {e}')
            try: cv2.destroyWindow(self.win)
            except: pass
            return None

# =========================================================
# 데이터/URDF 로드 및 맵
# =========================================================
print('[load] URDF ...')
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print(f"[WARN] RIGHT_HAND_URDF 로드 실패: {e}. 오른손 요소 생략.")
        right_hand = None

print('[load] data.json ...')
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

# 팔/손 매핑
l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
# 오른손 키 이름은 실제 data['info']['joint_names']['right_hand'] 확인 후 맞춰주세요.
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

# palm / tip 링크명 (URDF에 맞게 수정 가능)
L_PALM_LINK = 'L_hand_base_link'   # 좌손 바닥 링크
R_PALM_LINK = 'R_hand_base_link'   # 우손 바닥 링크 (파일 확인 필요)
L_TIP_LINK  = 'L_index_tip'
R_TIP_LINK  = 'R_index_tip'

def safe_q(names_list, qpos_list, mapping):
    q = {}
    for n, v in zip(names_list, qpos_list):
        if n in mapping:
            q[mapping[n]] = v
    return q

# =========================================================
# BODY URDF에서 wrist→hand 장착 변환 자동 추출
# =========================================================
def find_mount_transform(robot, wrist_link, prefer_side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            child_low = j.child.lower()
            if ('hand' in child_low) or ('finger' in child_low):
                cand.append(j)
    if not cand:
        return None, None
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            if prefer_side == 'left' and ('l_' in j.child.lower() or 'left' in j.child.lower()):
                picked = j; break
            if prefer_side == 'right' and ('r_' in j.child.lower() or 'right' in j.child.lower()):
                picked = j; break
    return picked, picked.origin  # origin은 4x4 동차변환

jL, T_wrist_to_hand_L = find_mount_transform(robot, 'left_wrist_yaw_link', 'left')
jR, T_wrist_to_hand_R = find_mount_transform(robot, 'right_wrist_yaw_link', 'right')

if T_wrist_to_hand_L is None:
    print('[WARN] left wrist->hand mount not found. Use Identity.')
    T_wrist_to_hand_L = np.eye(4)
else:
    print('[MOUNT-LEFT] parent:', jL.parent, 'child:', jL.child)

if T_wrist_to_hand_R is None:
    print('[WARN] right wrist->hand mount not found. Use Identity.')
    T_wrist_to_hand_R = np.eye(4)
else:
    print('[MOUNT-RIGHT] parent:', jR.parent, 'child:', jR.child)

# =========================================================
# FK
# =========================================================
def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = { amap[n]: v for n,v in zip(names, qpos) if n in amap }
    fk = robot.link_fk(q)

    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    Rw = T_wrist[:3,:3]
    pw = T_wrist[:3,3]
    return Rw, pw

def left_index_tip_world(frame, d_off=dL, dRvec=dRvec_L, dT=dT_L):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_q(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    if L_TIP_LINK not in left_hand.link_map:
        return None
    T_hand_tip = fk_h[left_hand.link_map[L_TIP_LINK]]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

def right_index_tip_world(frame, d_off=dR, dRvec=dRvec_R, dT=dT_R):
    if right_hand is None or (R_TIP_LINK not in getattr(right_hand, 'link_map', {})):
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_q(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)
    T_hand_tip = fk_h[right_hand.link_map[R_TIP_LINK]]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

# ---------- Palm (base) ----------
def left_palm_world(frame, d_off=dL, palm_off=pPalm_L):
    """손목 → (오프셋) → wrist->hand mount → hand base → palm center."""
    Rw, pw = wrist_pose_world(frame, 'left')
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_off = np.eye(4); T_off[:3,3] = d_off
    T_mount = T_wrist_to_hand_L
    T_palm = np.eye(4); T_palm[:3,3] = palm_off
    T_world_palm = T_world_wrist @ T_off @ T_mount @ T_palm
    return T_world_palm[:3,3]

def right_palm_world(frame, d_off=dR, palm_off=pPalm_R):
    if right_hand is None:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_off = np.eye(4); T_off[:3,3] = d_off
    T_mount = T_wrist_to_hand_R
    T_palm = np.eye(4); T_palm[:3,3] = palm_off
    T_world_palm = T_world_wrist @ T_off @ T_mount @ T_palm
    return T_world_palm[:3,3]

# =========================================================
# 클릭 수집/불러오기 (손목/손끝/손바닥)
# =========================================================
# 항목 키: LW, RW, LT, RT, LP, RP
def collect_clicks(frames):
    if LOAD_CLICKS and Path(CLICKS_JSON).exists():
        print(f'[load clicks] {CLICKS_JSON}')
        with open(CLICKS_JSON, 'r') as f:
            raw = json.load(f)
        return clicks_from_json(raw)

    clicker = Clicker(scale=1.0)
    clicks = {}
    for fr in frames:
        entry = {'LW':None,'RW':None,'LT':None,'RT':None,'LP':None,'RP':None}
        img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
        print(f'\n[frame {fr}] 클릭 수집 시작')
        entry['LW'] = clicker.click_or_none(img_path, f'{fr}-LW(wrist-left)')
        entry['RW'] = clicker.click_or_none(img_path, f'{fr}-RW(wrist-right)')
        entry['LT'] = clicker.click_or_none(img_path, f'{fr}-LT(left tip)')
        entry['RT'] = clicker.click_or_none(img_path, f'{fr}-RT(right tip)')
        entry['LP'] = clicker.click_or_none(img_path, f'{fr}-LP(left palm)')
        entry['RP'] = clicker.click_or_none(img_path, f'{fr}-RP(right palm)')

        # 내부에는 numpy로 저장 (계산 편의)
        for k in entry:
            if entry[k] is not None:
                entry[k] = np.asarray(entry[k], dtype=float)

        clicks[str(fr)] = entry

    if SAVE_CLICKS:
        with open(CLICKS_JSON, 'w', encoding='utf-8') as f:
            json.dump(clicks_to_jsonable(clicks), f, indent=2, ensure_ascii=False)
        print(f'[save clicks] {CLICKS_JSON}')
    return clicks

# =========================================================
# (선택) palm 오프셋만 간단 보정
# =========================================================
def refine_palm_offset(side, frames, clicks_dict):
    """손바닥 중심 pPalm_* (hand base frame) 3D만 추정"""
    if side == 'left':
        p0 = pPalm_L.copy()
        def palm_world(fr, p):
            return left_palm_world(fr, d_off=dL, palm_off=p)
        key = 'LP'
    else:
        if right_hand is None:
            print('[refine palm] right hand 없음. skip.')
            return np.zeros(3)
        p0 = pPalm_R.copy()
        def palm_world(fr, p):
            return right_palm_world(fr, d_off=dR, palm_off=p)
        key = 'RP'

    obs_list = []
    for fr in frames:
        ent = clicks_dict.get(str(fr), {})
        uv = ent.get(key, None)
        if uv is None:
            continue
        obs_list.append((fr, np.array(uv, dtype=float)))

    if len(obs_list) < 3:
        print(f'[refine palm {side}] 관측 {len(obs_list)} < 3  skip.')
        return np.zeros(3)

    def resid(p):
        res = []
        for fr, uv_gt in obs_list:
            pw = palm_world(fr, p)
            uv, pc = project_point_world(pw)
            if uv is None:
                res += [1000.0, 1000.0]
            else:
                res += [uv[0]-uv_gt[0], uv[1]-uv_gt[1]]
        return np.array(res, dtype=float)

    r = least_squares(resid, p0, verbose=1, ftol=1e-9, xtol=1e-9, gtol=1e-9, max_nfev=200,
                      loss='soft_l1', f_scale=3.0)
    print(f'[refine palm {side}] status:', r.status, r.message, ' cost:', r.cost)
    res_err = resid(r.x).reshape(-1,2)
    e2 = np.linalg.norm(res_err, axis=1)
    print(f'[refine palm {side}] reproj mean/std(px):', e2.mean(), e2.std())
    return r.x - p0   # delta 반환

# =========================================================
# 메인: 시각화
# =========================================================
def main():
    Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

    clicks = collect_clicks(FRAMES_VIS)

    # (선택) palm 오프셋만 간단 보정 원하면 주석 해제
    # dPalm_L = refine_palm_offset('left',  FRAMES_VIS, clicks)
    # dPalm_R = refine_palm_offset('right', FRAMES_VIS, clicks)
    # global pPalm_L, pPalm_R
    # pPalm_L = pPalm_L + dPalm_L
    # pPalm_R = pPalm_R + dPalm_R

    for fr in FRAMES_VIS:
        img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
        img = cv2.imread(img_path)
        if img is None:
            print('[WARN] image not found:', img_path)
            continue

        vis = img.copy()

        # wrist (오프셋 포함)
        RwL, pwL = wrist_pose_world(fr, 'left')
        RwR, pwR = wrist_pose_world(fr, 'right')
        p_wL = (RwL @ dL) + pwL
        p_wR = (RwR @ dR) + pwR
        uv_wL, _ = project_point_world(p_wL)
        uv_wR, _ = project_point_world(p_wR)

        # tip
        # p_tL = left_index_tip_world(fr)
        # p_tR = right_index_tip_world(fr)
        # uv_tL, _ = project_point_world(p_tL) if p_tL is not None else (None, None)
        # uv_tR, _ = project_point_world(p_tR) if p_tR is not None else (None, None)

        # palm
        p_pL = left_palm_world(fr, palm_off=pPalm_L)
        p_pR = right_palm_world(fr, palm_off=pPalm_R)
        uv_pL, _ = project_point_world(p_pL) if p_pL is not None else (None, None)
        uv_pR, _ = project_point_world(p_pR) if p_pR is not None else (None, None)

        # draw
        if uv_wL is not None:
            cv2.circle(vis, uv_wL, 8, COL_LW, -1); cv2.putText(vis, 'LW', (uv_wL[0]+6, uv_wL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_LW, 2)
        if uv_wR is not None:
            cv2.circle(vis, uv_wR, 8, COL_RW, -1); cv2.putText(vis, 'RW', (uv_wR[0]+6, uv_wR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_RW, 2)
        if uv_tL is not None:
            cv2.circle(vis, uv_tL, 7, COL_LT, -1); cv2.putText(vis, 'L_tip', (uv_tL[0]+6, uv_tL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_LT, 2)
        if uv_tR is not None:
            cv2.circle(vis, uv_tR, 7, COL_RT, -1); cv2.putText(vis, 'R_tip', (uv_tR[0]+6, uv_tR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_RT, 2)
        if uv_pL is not None:
            cv2.circle(vis, uv_pL, 7, COL_LP, -1); cv2.putText(vis, 'L_palm', (uv_pL[0]+6, uv_pL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_LP, 2)
        if uv_pR is not None:
            cv2.circle(vis, uv_pR, 7, COL_RP, -1); cv2.putText(vis, 'R_palm', (uv_pR[0]+6, uv_pR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, COL_RP, 2)

        cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)

        out_path = str(Path(OUT_DIR) / f'vis_{fr}.png')
        cv2.imwrite(out_path, vis)
        print('Saved:', out_path)

    print('\n완료!')
    print('- palm이 크게 어긋나면 pPalm_L/R을 약간 조정하고 다시 확인해봐.')
    print('- tip이 어긋나면 dRvec/dT (장착 회전/이동 보정)도 조정 가능.')
    print('- GUI가 안 뜨면 USE_GUI=False로 두고, CLICKS_JSON에 수동으로 픽셀 좌표를 채워 넣어도 돼.')

if __name__ == '__main__':
    main()


[load] URDF ...
[load] data.json ...
[MOUNT-LEFT] parent: left_wrist_yaw_link child: left_hand_palm_link
[MOUNT-RIGHT] parent: right_wrist_yaw_link child: right_hand_palm_link

[frame 0] 클릭 수집 시작
[ 0-LW(wrist-left) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s 또는 ESC 건너뛰기
[ 0-RW(wrist-right) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s 또는 ESC 건너뛰기
[ 0-LT(left tip) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s 또는 ESC 건너뛰기
[ 0-RT(right tip) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s 또는 ESC 건너뛰기
[ 0-LP(left palm) ] /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg — 좌클릭(선택) / s